<a href="https://colab.research.google.com/github/prat-hart/chess_game/blob/main/ChessGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Learning Objectives



*   Game Playing
*   Chess - Board Setup & Rules
*   Adversarial Search
*   AI - Random vs MinMax


##Import Libraries

The code here will contain only **import** statements. A base list of the required libraries are already imported. You will most likely not need any other libraries, but if needed, add the import statements here. As mentioned before, you can not use any premade chess libraries.

In [ ]:
import numpy as np
import string
import random
import os
import sys
import time
from IPython.display import clear_output

In [ ]:
def ChessBoardSetup():
#black=lower case 
  coordinates = [[(i,j) for i in range(0,8)] for j in range(0,8)]
  state = ["rtbqkbtr", "pppppppp", "........", "........", "........","........", "PPPPPPPP", "RTBQKBTR"]
  state = np.asarray([list(r) for r in state])
  return state

def DrawBoard(state):
  for r in state:
      print(' '.join(r))
      
def MovePiece(board_state,start,destination):
    state=board_state.copy()
    state[destination[0],destination[1]]=state[start[0],start[1]]
    state[start[0],start[1]]= '.'
    return state

##ChessRules

In [ ]:
def IsMoveLegal(board_state,start,destination,hypothetical=False):
      return destination in GetListOfLegalMoves(board_state,start,hypothetical)
      
def GetListOfLegalMoves(state,start,hypothetical=False):
  #main rules function
  #dictionary of all possible moves for each Black and White piece 
  moves= {'p': lambda x:[[x[0]+1, x[1]], [x[0]+2, x[1]],[x[0]+1,x[1]+1], [x[0]+1, x[1]-1]],
     'P': lambda x:[[x[0]-1, x[1]], [x[0]-2, x[1]], [x[0]-1,x[1]-1], [x[0]-1, x[1]+1]],
     'k': lambda x:[[x[0]+1, x[1]+1], [x[0]-1, x[1]-1], [x[0]-1, x[1]+1], [x[0]+1, x[1]-1],[x[0]+1, x[1]], [x[0]-1, x[1]],[x[0], x[1]+1], [x[0], x[1]-1]],
     'K': lambda x:[[x[0]+1, x[1]+1], [x[0]-1, x[1]-1], [x[0]-1, x[1]+1], [x[0]+1, x[1]-1],[x[0]+1, x[1]], [x[0]-1, x[1]],[x[0], x[1]+1], [x[0], x[1]-1]],
     'r': lambda x:[[x[0], i] for i in range(state.shape[1]) if i != x[1]] + [[i, x[1]] for i in range(state.shape[0]) if i != x[0]],
     'R': lambda x:[[x[0], i] for i in range(state.shape[1]) if i != x[1]] + [[i, x[1]] for i in range(state.shape[0]) if i != x[0]],
     'b': lambda x:[[i,x[1]-(x[0]-i)] for i in range(state.shape[1]) if i != x[0]] + [[i,x[1]+(x[0]-i)] for i in range(state.shape[1]) if i != x[0]],
     'B': lambda x:[[i,x[1]-(x[0]-i)] for i in range(state.shape[1]) if i != x[0]] + [[i,x[1]+(x[0]-i)] for i in range(state.shape[1]) if i != x[0]],
     't': lambda x:[[x[0]+2, x[1]+1], [x[0]+1, x[1]+2], [x[0]-1, x[1]+2], [x[0]-2, x[1]+1], [x[0]-2, x[1]-1], [x[0]-1, x[1]-2], [x[0]+1, x[1]-2], [x[0]+2, x[1]-1]],
     'T': lambda x:[[x[0]+2, x[1]+1], [x[0]+1, x[1]+2], [x[0]-1, x[1]+2], [x[0]-2, x[1]+1], [x[0]-2, x[1]-1], [x[0]-1, x[1]-2], [x[0]+1, x[1]-2], [x[0]+2, x[1]-1]]}

  piece= state[start[0],start[1]]

  #adds the Rook and Bishop rules towards the Queen's moving capabilities
  if piece== '.':
    return []
  if piece.upper() == 'Q':
    poss_move= moves['r'](start)+moves['b'](start)
  else: 
    poss_move= moves[piece](start)
 
  #pawn cases, down-selecting 
  if piece.upper() == 'P':
    if piece == 'p' and start[0] != 1 :
      del poss_move[1]
    elif piece == 'P' and start[0] != 6:
      del poss_move[1]
    try:
      if state[poss_move[0][0],poss_move[0][1]] != '.':
        del poss_move[0]
    except:
        del poss_move[0]
    try:
      if state[poss_move[-2][0],poss_move[-2][1]]=='.' or state[poss_move[-2][0],poss_move[-2][1]].islower() == piece.islower():
        del poss_move[-2]
    except:
      del poss_move[-2]
    try:
      if state[poss_move[-1][0],poss_move[-1][1]]=='.' or state[poss_move[-1][0],poss_move[-1][1]].islower() == piece.islower():
        del poss_move[-1]
    except:
      del poss_move[-1]

  #restricts possible movements to 8x8 board
  on_board=[x for x in poss_move if x[0]>= 0 and x[0]<=7 and x[1]>=0 and x[1]<= 7]

  #case:own piece is blocking movement
  not_blocked= [x for x in on_board if state[x[0],x[1]]=='.' or state[x[0],x[1]].islower() != piece.islower()]
  if piece.upper() in ['P', 'Q', 'B', 'R']:
    not_blocked= [x for x in not_blocked if IsClearPath(state,start,x)]
  if hypothetical:
    return not_blocked

  #case: restricts if player in check
  player='black' if piece.islower() else 'white'
  not_in_check=[x for x in not_blocked if not DoesMovePutPlayerInCheck(state,player,start,x)]
  return not_in_check

def GetPiecesWithLegalMoves(board_state,player):
  # gets a list of all pieces for the current player that have legal moves 
  if player== 'black':
    black_pieces= ['b','r','t','p','q','k']
    black= np.isin(board_state,black_pieces)
    pieces= np.array(np.nonzero(black)).T

  elif player=='white':
    white_pieces= ['B','R','T','P','Q','K']
    white= np.isin(board_state, white_pieces)
    pieces= np.array(np.nonzero(white)).T

  moves=[GetListOfLegalMoves(board_state,x) for x in pieces]
  list_of_pieces=[x.tolist() for i,x in enumerate(pieces) if moves[i]]
  return list_of_pieces,[m for m in moves if m]

def IsCheckmate(board_state,player):
  # returns True if the current player is in checkmate, else False
  return IsInCheck(board_state,player) and GetPiecesWithLegalMoves(board_state,player)[0]==[]

def IsInCheck(board_state, player):
  if player == 'black':
    black_king=['k']
    check_black= board_state == black_king
    destination= np.array(np.nonzero(check_black)).T.tolist()[0]
    white_pieces= ['B','R','T','P','Q']
    white= np.isin(board_state, white_pieces)
    enemy_pieces= np.array(np.nonzero(white)).T
  else:
    white_king=['K']
    check_white= board_state == white_king
    destination= np.array(np.nonzero(check_white)).T.tolist()[0]
    black_pieces= ['b','r','t','p','q']
    black= np.isin(board_state,black_pieces)
    enemy_pieces= np.array(np.nonzero(black)).T

  is_in_check= [x for x in enemy_pieces if IsMoveLegal(board_state,x,destination,hypothetical=True)]
  return is_in_check != []
  
def IsClearPath(board_state,start,destination):  
  #checks path between start and destination for any pieces in the way
  if start[0] <= destination[0]:
    rows= list(range(start[0]+1, destination[0]))
    if start[1] <= destination[1]:
      cols= list(range(start[1]+1,destination[1]))
    elif start[1] > destination[1]:
      cols= list(range(start[1]-1,destination[1],-1))
  elif start[0] > destination[0]:
    rows= list(range(destination[0]+1,start[0]))
    if start[1] <= destination[1]:
      cols= list(range(destination[1]-1,start[1],-1))
    elif start[1] > destination[1]:
      cols= list(range(start[1]+1,destination[1]))
  
  if not rows:
    rows=[start[0] for i in range(len(cols))]
  if not cols:
    cols=[start[1] for i in range(len(rows))]
  return (board_state[rows,cols]== ".").all()
  
  
def DoesMovePutPlayerInCheck(some_state,player, start, destination):
    # returns True if it puts current player into check
    state= MovePiece(some_state, start,destination)
    return IsInCheck(state,player)
    

##RandomAI



In [ ]:
def GetRandomMove(board_state,player):
  # pick a random piece and a random legal move for that piece
  pieces,moves= GetPiecesWithLegalMoves(board_state,player)
  i = np.random.randint(0,len(pieces)) if type(pieces[0]) == list else 0
  piece= pieces[i]
  moves= moves[i]
  i= np.random.randint(0,len(moves)) if type(moves[0]) == list else 0
  move= moves[i]
  return piece,move

##MinMaxAI

In [ ]:
def evaluation(board_state):
  #evaluates piece cost for each and factors in the amount of legal moves for each potential piece
  #more movment possiblities, better score

  piece_cost={'p':1,'r':5,'t':3,'b':3,'q':9,'P':-1,'R':-5,'T':-3,'B':-3,'Q':-9, 'k':100, 'K':-100, '.':0}
  compute_cost = np.vectorize(lambda x: piece_cost[x])
  board_total=compute_cost(board_state).sum()
  
  pieces_b,moves=GetPiecesWithLegalMoves(board_state,'black')
  board_total+=sum([len(m) for m in moves])

  pieces_w,moves=GetPiecesWithLegalMoves(board_state,'white')
  board_total-=sum([len(m) for m in moves])

  return board_total

def GetMinMaxMove(board_state,player,depth=2,cur_move=[0,0]):
  pieces,destinations= GetPiecesWithLegalMoves(board_state,player)

  if depth==0 or pieces==[]:
    return evaluation(board_state),cur_move

  if player == 'black':
    best_score=-1e6
    best_move=cur_move
    for i,piece in enumerate(pieces):
      for destination in destinations[i]:
        moves=MovePiece(board_state,piece,destination)
        scores,move=GetMinMaxMove(moves,'white',depth-1,[piece,destination])
        best_score=max(best_score,scores)
        best_move=[piece,destination] if best_score == scores else best_move
    return best_score,best_move
    
  elif player == 'white':
    best_score=1e6
    best_move=cur_move
    for i,piece in enumerate(pieces):
      for destination in destinations[i]:
        moves=MovePiece(board_state,piece,destination)
        scores,move=GetMinMaxMove(moves,'black',depth-1,[piece,destination])
        best_score=min(best_score,scores)
        best_move=[piece,destination] if best_score == scores else best_move
    return best_score,best_move
    
def GetAlphaBeta(board_state,player,alpha=-1e6,beta=1e6,depth=2,cur_move=[0,0]):
  #extra credit MINMAX with Alpha Beta Pruning
  pieces,destinations= GetPiecesWithLegalMoves(board_state,player)

  if depth==0 or pieces==[]:
    return evaluation(board_state),cur_move

  if player == 'black':
    best_score=-1e6
    best_move=cur_move
    for i,piece in enumerate(pieces):
      for destination in destinations[i]:
        moves=MovePiece(board_state,piece,destination)
        scores,move=GetAlphaBeta(moves,'white',alpha,beta,depth-1,[piece,destination])
        best_score=max(best_score,scores)
        alpha=max(alpha,best_score)
        best_move=[piece,destination] if best_score == scores else best_move
        if best_score >= beta:
          break
    return best_score,best_move
    
  elif player == 'white':
    best_score=1e6
    best_move=cur_move
    for i,piece in enumerate(pieces):
      for destination in destinations[i]:
        moves=MovePiece(board_state,piece,destination)
        scores,move=GetAlphaBeta(moves,'black',alpha,beta,depth-1,[piece,destination])
        best_score=min(best_score,scores)
        beta=min(beta,best_score)
        best_move=[piece,destination] if best_score == scores else best_move
        if best_score <= alpha:
          break
    return best_score,best_move


#Game Setup & Main Loop

(10 points)

Write code below to have a game-play between two AIs - Random vs MinMax. For each iteration, draw the board before and after a turn. 

In [ ]:
state=ChessBoardSetup()
DrawBoard(state)
player='white'
black_moves=[]
white_moves=[]
pieces=[]
turn=0
N=25

while not IsCheckmate(state,player) and turn < N:
  if player=='white':
    move=GetRandomMove(state,player)
    white_moves.append(move)
    piece=state[move[0][0],move[0][1]]
    state=MovePiece(state,move[0],move[1])
    player='black'
    time.sleep(1)
  else:
    score,move=GetAlphaBeta(state,player)
    piece=state[move[0][0],move[0][1]]
    state=MovePiece(state,move[0],move[1])
    black_moves.append(move)
    player='white'

  clear_output()
  DrawBoard(state)
  turn+=0.5
  pieces.append(piece)
  print(f'turn: {turn}')
  print(f'Log of White Moves: {white_moves}')
  print(f'Log of Black Moves: {black_moves}')
  print(f'Acting Pieces: {pieces}')

# check and print - Stalemate or Checkmate
if IsCheckmate(state,player)==True:
  print('Checkmate')
elif GetListOfLegalMoves(state,move[1])==[]:
  print('Stalemate')

r . . . k . . .
p p p t . p . p
. . . . . . . p
P . . . b t . .
. . b . K r . .
. . . . . p . .
. . . . . P . P
R . . . . . . R
turn: 24.0
Log of White Moves: [([6, 6], [4, 6]), ([6, 4], [4, 4]), ([6, 3], [4, 3]), ([7, 3], [6, 3]), ([7, 5], [6, 4]), ([6, 1], [4, 1]), ([6, 3], [2, 7]), ([4, 3], [3, 4]), ([7, 1], [5, 2]), ([7, 4], [7, 3]), ([3, 4], [2, 4]), ([6, 4], [5, 5]), ([7, 3], [6, 4]), ([6, 0], [4, 0]), ([6, 4], [5, 4]), ([7, 0], [5, 0]), ([7, 6], [6, 4]), ([4, 0], [3, 0]), ([5, 4], [6, 3]), ([6, 3], [5, 4]), ([6, 2], [4, 2]), ([5, 4], [4, 4]), ([5, 0], [7, 0]), ([7, 2], [4, 5])]
Log of Black Moves: [[[1, 4], [3, 4]], [[0, 3], [2, 5]], [[2, 5], [2, 4]], [[2, 4], [4, 6]], [[4, 6], [2, 4]], [[2, 4], [2, 7]], [[1, 6], [2, 7]], [[0, 5], [4, 1]], [[4, 1], [5, 2]], [[1, 3], [3, 3]], [[0, 2], [2, 4]], [[0, 1], [1, 3]], [[3, 3], [4, 4]], [[2, 4], [4, 2]], [[4, 4], [5, 5]], [[4, 2], [2, 0]], [[2, 0], [6, 4]], [[5, 2], [3, 4]], [[0, 6], [1, 4]], [[0, 7], [0, 6]], [[1, 4], [3, 5]], [[6, 4], 